# MSRA初始化
“Xavier” 是一种相对不错的初始化方法，在博文[“深度学习——Xavier初始化方法”](https://blog.csdn.net/shuzfan/article/details/51338178)中有介绍。但是，Xavier推导的时候假设激活函数是线性的，显然我们目前常用的ReLU和PReLU并不满足这一条件。

只考虑输入个数时，[MSRA](https://blog.csdn.net/shuzfan/article/details/51347572) 初始化是一个均值为`0`方差为`2/n`的高斯分布：

# [【CV知识学习】神经网络梯度与归一化问题总结+highway network、ResNet的思考](http://www.cnblogs.com/jie-dcai/p/5803220.html)

可以看到，当网络加深，训练的误差反而上升了，而加入了highway之后，这个问题得到了缓解。一般来说，深度网络训练困难是由于梯度回流受阻的问题，可能浅层网络没有办法得到调整，或者我自己YY的一个原因是（回流的信息经过网络之后已经变形了，很可能就出现了internal covariate shift类似的问题了）。Highway Network 受LSTM启发，增加了一个门函数，让网络的输出由两部分组成，分别是网络的直接输入以及输入变形后的部分。

In [1]:
import sys
sys.path.append('E:/xlab')


from mxnet.gluon import nn
from mxnet import nd, autograd, init
from gluoncv.model_zoo import get_model
import tables as tb

from base.xloader import Loader


xpath = 'E:/xdata/X.h5'
h5 = tb.open_file(xpath)
cifar = h5.root.cifar10

batch_size = 256
train_data = Loader(cifar.trainX, cifar.trainY, batch_size, True)
test_data = Loader(cifar.testX, cifar.testY, batch_size, False)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pnet = get_model('cifar_resnet20_v2', pretrained = True)

In [5]:
for imgs, labels in iter(train_data):
    X = nd.array(imgs)
    break

In [22]:
x = nd.random.normal(shape=(10, 3, 64, 64))
xnet.features(x)


[[0.00336762 0.01516689 0.00253347 ... 0.00504274 0.00964877 0.05146184]
 [0.00439547 0.02479693 0.         ... 0.12292846 0.0666824  0.00936293]
 [0.0187403  0.02740918 0.01029928 ... 0.         0.02759702 0.02019245]
 ...
 [0.0615379  0.08300509 0.         ... 0.01976576 0.09429143 0.        ]
 [0.03057484 0.02722096 0.         ... 0.01825439 0.06992494 0.10787252]
 [0.06040259 0.05962146 0.01497491 ... 0.01512903 0.0538373  0.01285281]]
<NDArray 10x256 @cpu(0)>

In [36]:
class XNet(nn.HybridSequential):
    
    def __init__(self, classes = 100, **kwargs):
        super().__init__(**kwargs)
        self.features = get_model('cifar_resnet20_v2', classes = 10).features
        self.features.add(
            nn.Dense(classes)
        )
        
    def hybird_forward(self, F, x):
        x = self.features(x)
        return x

In [37]:
net = XNet()
net.initialize(init.Xavier(magnitude=2))
net.hybridize()